# MTS on GPU

In [3]:
import numpy as np
import cupy as cp
import random
from collections import deque
import time
import json
import os

print("Libraries loaded. Checking GPU...")
try:
    print(f"CuPy Device: {cp.cuda.Device(0).pci_bus_id}")
except Exception as e:
    print(f"GPU Error: {e}. Is the runtime set to GPU?")

ModuleNotFoundError: No module named 'cupy'

In [4]:
def calculate_labs_energy_batch_gpu(sequences):
    """
    Calculates the LABS energy for a batch of binary sequences using CuPy.
    Args:
        sequences (cupy.ndarray): Binary sequences of shape (batch_size, N) with values {-1, 1}.
    Returns:
        cupy.ndarray: Energy values for each sequence in the batch.
    """
    if not isinstance(sequences, cp.ndarray):
        sequences = cp.array(sequences)

    batch_size, N = sequences.shape
    energies = cp.zeros(batch_size, dtype=cp.float32)

    # Vectorized autocorrelation calculation
    for k in range(1, N):
        # products shape: (batch_size, N-k)
        products = sequences[:, :N-k] * sequences[:, k:]
        C_k = cp.sum(products, axis=1)
        energies += C_k ** 2

    return energies

def calculate_labs_energy_single_gpu(sequence):
    if not isinstance(sequence, cp.ndarray):
        sequence = cp.array(sequence)
    return float(calculate_labs_energy_batch_gpu(sequence.reshape(1, -1))[0])

def batch_flip_neighbors_gpu(current_sequences):
    """
    Generates all single-bit flip neighbors for a batch of sequences.
    Returns shape (batch_size * N, N)
    """
    batch_size, N = current_sequences.shape

    # Create copies: (batch_size, N, N)
    neighbors = cp.tile(current_sequences[:, cp.newaxis, :], (1, N, 1))

    # Flip mask with -1 on diagonal
    flip_mask = cp.ones((N, N), dtype=cp.int8)
    cp.fill_diagonal(flip_mask, -1)

    # Apply flips
    all_neighbors = current_sequences[:, cp.newaxis, :] * flip_mask[cp.newaxis, :, :]

    return all_neighbors.reshape(-1, N)

In [5]:
def initialize_population(N, pop_size):
    return [np.random.choice([-1, 1], size=N) for _ in range(pop_size)]

def combine_bitstrings(s1, s2):
    N = len(s1)
    crossover_point = random.randint(1, N - 1)
    return np.concatenate((s1[:crossover_point], s2[crossover_point:]))

def mutate_bitstring(s, p_mutate):
    mutated = np.copy(s)
    mask = np.random.random(len(mutated)) < p_mutate
    mutated[mask] *= -1
    return mutated

def tabu_search_gpu(start_sequence, N_tabu, max_iterations):
    """GPU Accelerated Tabu Search local optimization"""
    N = len(start_sequence)
    current = np.copy(start_sequence)

    # Initial energy (GPU)
    current_energy = calculate_labs_energy_single_gpu(current)

    best = np.copy(current)
    best_energy = current_energy

    tabu_list = deque(maxlen=N_tabu)

    for _ in range(max_iterations):
        # 1. Generate neighbors (GPU)
        current_gpu = cp.array(current).reshape(1, -1)
        neighbors_gpu = batch_flip_neighbors_gpu(current_gpu)

        # 2. Evaluate energies (GPU)
        neighbor_energies_gpu = calculate_labs_energy_batch_gpu(neighbors_gpu)
        neighbor_energies = cp.asnumpy(neighbor_energies_gpu)

        # 3. Selection (CPU Logic)
        sorted_indices = np.argsort(neighbor_energies)

        found_move = False
        best_neighbor_idx = -1
        best_neighbor_energy = float('inf')

        for idx in sorted_indices:
            energy = neighbor_energies[idx]
            is_tabu = idx in tabu_list
            # Aspiration criteria: allow tabu move if it improves global best
            is_better_than_best = energy < best_energy

            if not is_tabu or is_better_than_best:
                best_neighbor_idx = idx
                best_neighbor_energy = energy
                found_move = True
                break

        if found_move:
            current[best_neighbor_idx] *= -1
            current_energy = best_neighbor_energy
            tabu_list.append(best_neighbor_idx)

            if current_energy < best_energy:
                best_energy = current_energy
                best = np.copy(current)
        else:
            break

    return best, best_energy

def memetic_tabu_search_gpu(N, pop_size, p_mutate, N_tabu, max_iterations, tabu_iterations):
    """Full GPU-Enhanced MTS Loop"""
    population = initialize_population(N, pop_size)

    # Initial Population Evaluation (GPU)
    pop_gpu = cp.array(population)
    energies_gpu = calculate_labs_energy_batch_gpu(pop_gpu)
    energies = cp.asnumpy(energies_gpu).tolist()

    best_idx = np.argmin(energies)
    best_energy = energies[best_idx]
    best_sequence = np.copy(population[best_idx])

    for iteration in range(max_iterations):
        # Crossover & Mutation
        parents = random.sample(range(pop_size), 2)
        parent1, parent2 = population[parents[0]], population[parents[1]]
        child = combine_bitstrings(parent1, parent2)
        child = mutate_bitstring(child, p_mutate)

        # Local Search (GPU)
        optimized_child, child_energy = tabu_search_gpu(child, N_tabu, tabu_iterations)

        # Replacement Strategy
        worst_idx = np.argmax(energies)
        if child_energy < energies[worst_idx]:
            population[worst_idx] = optimized_child
            energies[worst_idx] = child_energy

        if child_energy < best_energy:
            best_energy = child_energy
            best_sequence = np.copy(optimized_child)

    return best_sequence, best_energy

In [6]:
def run_benchmark_cycle(N, pop_size=100, p_mutate=0.2, N_tabu=10, max_iterations=200, tabu_iterations=50):
    print(f"\n>>> Running Benchmark for N={N}...")
    start_time = time.time()

    best_seq, best_energy = memetic_tabu_search_gpu(
        N, pop_size, p_mutate, N_tabu, max_iterations, tabu_iterations
    )

    duration = time.time() - start_time
    print(f"  Done in {duration:.4f}s | Best Energy: {best_energy}")
    return {"N": N, "time": duration, "energy": best_energy}

# --- Benchmark Settings ---
n_values = [20, 30, 40, 50]
results = []

for N in n_values:
    try:
        res = run_benchmark_cycle(N)
        results.append(res)
    except Exception as e:
        print(f"Error at N={N}: {e}")

print("\n=== Final Results ===")
for r in results:
    print(f"N={r['N']}: Time={r['time']:.2f}s, Energy={r['energy']}")


>>> Running Benchmark for N=20...
Error at N=20: name 'time' is not defined

>>> Running Benchmark for N=30...
Error at N=30: name 'time' is not defined

>>> Running Benchmark for N=40...
Error at N=40: name 'time' is not defined

>>> Running Benchmark for N=50...
Error at N=50: name 'time' is not defined

=== Final Results ===


In [7]:
# --- Benchmark Settings ---
n_values = [5, 10, 20, 30, 40, 50, 70, 100]
results = []

for N in n_values:
    try:
        res = run_benchmark_cycle(N)
        results.append(res)
    except Exception as e:
        print(f"Error at N={N}: {e}")

print("\n=== Final Results ===")
for r in results:
    print(f"N={r['N']}: Time={r['time']:.2f}s, Energy={r['energy']}")


>>> Running Benchmark for N=5...
Error at N=5: name 'time' is not defined

>>> Running Benchmark for N=10...
Error at N=10: name 'time' is not defined

>>> Running Benchmark for N=20...
Error at N=20: name 'time' is not defined

>>> Running Benchmark for N=30...
Error at N=30: name 'time' is not defined

>>> Running Benchmark for N=40...
Error at N=40: name 'time' is not defined

>>> Running Benchmark for N=50...
Error at N=50: name 'time' is not defined

>>> Running Benchmark for N=70...
Error at N=70: name 'time' is not defined

>>> Running Benchmark for N=100...
Error at N=100: name 'time' is not defined

=== Final Results ===


In [8]:
%run run_all_benchmarks.py


Exception: File `'run_all_benchmarks.py'` not found.